# PySpark Cookbook

### Tomasz Drabas, Denny Lee
#### Version: 0.1
#### Date: 2/28/2018

# Loading the data

In [1]:
census_path = '../data/census_income.csv'

census = spark.read.csv(census_path, header=True, inferSchema=True)
census.count()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


SparkSession available as 'spark'.
32561

In [2]:
census.show()

+---+-----------------+------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|fnlwgt|    education|education-num|      marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loss|hours-per-week|native-country| label|
+---+-----------------+------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|        2174|           0|            40| United-States| <=50K|
| 50| Self-emp-not-inc| 83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|           0|           0|            1

In [3]:
census.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- label: string (nullable = true)

## Loading into RDD

In [34]:
census_rdd = sc.textFile(census_path)

header = census_rdd.first().split(',')
census_split = (
    census_rdd
    .map(lambda row: row.split(','))
    .filter(lambda row: row != header) # remove header
)

census_split.take(1)

[['39', ' State-gov', '77516', ' Bachelors', '13', ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', '2174', '0', '40', ' United-States', ' <=50K']]

# Transforming the data

Using hashing trick

In [37]:
import pyspark.mllib.feature as feat
import pyspark.mllib.regression as reg
import pyspark.mllib.linalg as ln

final_data = (
    census
    .rdd
    .map(lambda row: [
        list(
            feat.HashingTF(len_ftrs[col])
            .transform(row[i])
            .toArray()
        ) if i > 5 
        else [row[i]] 
        for i, col in enumerate(cols_to_keep)]
    )
    .map(lambda row: [item for sublist in row
                  for item in sublist])
#     .map(lambda row: reg.LabeledPoint(
#         row[0],
#         ln.Vectors.dense(row[1:]))
#     )
)

final_data.take(3)

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 34, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 1339, in takeUpToNumLeft
  File "<stdin>", line 15, in <lambda>
NameError: name 'cols_to_keep' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.ap

In [ ]:
# for col in str_cols:    
#     featNo = (
#         census
#         .select(col)
#         .distinct()
#         .count()
#     )

#     hashTrick = feat.HashingTF(featNo)

#     print(col, featNo)
#     census.withColumn('col_ht', hashTrick(F.col(col)))
#     encoded = (
#         census
#         .select(col)
#         .distinct()
#         .withColumn('encoding', 
#                     F.row_number()
#                     .over(
#                         W.Window
#                         .partitionBy()
#                         .orderBy(col)
#                     )
#                    )
#         .toPandas()
#         .to_dict('records')
#     )

#     encoding[col] = dict([(e[col], e['encoding']) for e in encoded])

In [ ]:
encoding

# Exploring the data

# Testing the data

# Creating an RDD for training

# Predicting a workclass of census respondents

# Forecasting income levels of census respondents

# Building clustering models

# Computing performance statistics